# Does the `--all-txs` flag help the fullnode to receive pending transactions faster?

The bloXroute gateway docker container has a `--all-txs` flag, which can make the local gateway send all transactions to the local fullnode via the P2P connection. Without this flag only 10-20% transactions are sent to the fullnode.


This notebook compares the effect of `--all-txs`.

Steps:

* First, start a local bloXroute gateway with the `--all-txs` flag, and run the [bloxroute_tx](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/bloxroute/tx/bloxroute_tx.go) and [fullnode_tx](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/fullnode/tx/fullnode_tx.go) to collect data
* Second, stop the local gateway, and restart it without the `--all-txs` flag, then run `bloxroute_tx` and `fullnode_tx` to collect data again
* Substract the `received_at` timestamp of txs with the same hash, we get to know the gap between with and without the flag.

Hardware:

* A machine with 128 cores, 256GB memory in a Fremont IDC
* A `m5zn.3xlarge` instance in AWS Virginia regon

Files:

* `bloxroute-newtxs-cloud-fremont-with-gateway-all-txs.json.gz`, `bloxroute-newtxs-cloud-virginia-with-gateway-all-txs.json.gz`, `bloxroute-newtxs-gateway-fremont-all-txs.json.gz`, `bloxroute-newtxs-gateway-virginia-all-txs.json.gz`, `fullnode-tx-fremont-with-gateway-all-txs.json.gz` and `fullnode-tx-virginia-with-gateway-all-txs.json.gz` are generated in step 1
* `bloxroute-newtxs-cloud-fremont-with-gateway.json.gz`, `fullnode-tx-fremont-with-gateway.json.gz` and `fullnode-tx-virginia-with-gateway.json.gz` are generated in step 2

In [1]:
import gzip
import json
import pandas as pd
from typing import Dict

In [2]:
pd.io.json._json.loads = lambda s, *a, **kw: json.loads(s)

In [3]:
def read_timestamp(file: str) -> Dict[str, int]:
    result: Dict[str, int] = {}  # hash -> received_at
    f_in = gzip.open(file, "rt") if file.endswith('.json.gz') else open(file, "rt")
    with f_in:
        for line in f_in:
            obj = json.loads(line)
            if 'received_at' in obj:
                if 'hash' in obj:  # fullnode
                    result[obj['hash']] = obj['received_at']
                elif 'txHash' in obj:  # bloXroute
                    result[obj['txHash']] = obj['received_at']
                else:
                    raise ValueError(line)
    return result

In [4]:
def compare_timestamp(file1: str, file2: str) -> pd.DataFrame:
    dict1 = read_timestamp(file1)
    dict2 = read_timestamp(file2)
    diff = {}
    for tx_hash, received_at in dict1.items():
        if tx_hash in dict2:
            diff[tx_hash] = received_at-dict2[tx_hash]
    
    df = pd.DataFrame(diff.items(), columns=['tx_hash', 'gap'])
    # remove outliers
    quantile05 = df['gap'].quantile(0.05)
    quantile95 = df['gap'].quantile(0.95)
    return df[(df['gap'] >= quantile05) & (df['gap']<=quantile95)]

## In Fremont IDC

In [5]:
compare_timestamp('./data/bloxroute-newtxs-cloud-fremont-with-gateway.json.gz', './data/fullnode-tx-fremont-with-gateway.json.gz').describe()

,gap
count,142864.000000
mean,-116.179996
std,174.895064
min,-612.000000
25%,-98.000000
50%,-52.000000
75%,-6.000000
max,26.000000


In [6]:
compare_timestamp('./data/bloxroute-newtxs-cloud-fremont-with-gateway-all-txs.json.gz', './data/fullnode-tx-fremont-with-gateway-all-txs.json.gz').describe()

,gap
count,154234.000000
mean,-0.026369
std,3.257345
min,-3.000000
25%,-2.000000
50%,-1.000000
75%,0.000000
max,19.000000


**Conclusion: The cloud API is 52ms faster than the fullnode without `--all-txs`, and 1ms faster than the fullnode with `--all-txs`, which means the fullnode is 51ms faster after the `--all-txs` flag is on.**

## In AWS Virginia

In [7]:
compare_timestamp('./data/bloxroute-newtxs-cloud-virginia-with-gateway.json.gz', './data/fullnode-tx-virginia-with-gateway.json.gz').describe()

,gap
count,143942.000000
mean,-214.868211
std,213.300921
min,-599.000000
25%,-443.000000
50%,-116.000000
75%,-10.000000
max,17.000000


In [8]:
compare_timestamp('./data/bloxroute-newtxs-cloud-virginia-with-gateway-all-txs.json.gz', './data/fullnode-tx-virginia-with-gateway-all-txs.json.gz').describe()

,gap
count,152327.000000
mean,0.919732
std,3.934429
min,-3.000000
25%,-1.000000
50%,0.000000
75%,1.000000
max,25.000000


**Conclusion: The cloud API is 116ms faster than the fullnode without `--all-txs`, and 0.0ms faster than the fullnode with `--all-txs`, which means the fullnode is 116ms faster after the `--all-txs` flag is on.**

## Conclusion

**Overall, the `--all-txs` flag significantly help the local fullnode sync faster.** Note one sideeffect is that the fullnode consumes more CPU and memory.

Gaps: Virginia gateway with `--all-txs` -> 1ms -> Virginia fullnode with gateway and `--all-txs` -> 0 -> Virginia Cloud -> 27ms -> Fremont gateway with `--all-txs` -> 0 -> Fremont Cloud -> 1ms -> Fremont fullnode with gateway and `--all-txs`